## Faster R-CNN Model Training

geschrieben von Benjamin H.<br>
Studiengang: Informatik - Software & Information Engineering<br>
6. Semester - Sommersemester 2022

### Installation von wichtigen Modulen für KI

In [ ]:
# PyTorch & TorchVision
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
# Cython & PyYAML
!pip install cython pyyaml==5.1
# COCOAPI
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# Detectron2
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

### Import von wichtigen Modulen

In [ ]:
!nvcc --version

In [ ]:
import copy
import cv2
import detectron2
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import random
import torch, torchvision

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import build_detection_train_loader, MetadataCatalog
from detectron2.data import detection_utils as utils
from detectron2.data import transforms as T
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

from google.colab import files
from google.colab.patches import cv2_imshow

from skimage.util import random_noise

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

### Download der Datensatzes aus GitHub

In [ ]:
!curl -L "https://app.roboflow.com/ds/2wDxw552ZJ?key=P7pSLXDYkB" > dataset.zip; unzip dataset.zip; rm dataset.zip

### Registrierung des Datensatzes an Detectron2

In [ ]:
DatasetCatalog.clear()
register_coco_instances("datasetTrain", {}, "/content/train/_annotations.coco.json", "/content/train")
register_coco_instances("datasetValidate", {}, "/content/valid/_annotations.coco.json", "/content/valid")

### Visualisierung der Trainingsdaten

In [ ]:
trainMetadata = MetadataCatalog.get("datasetTrain")
trainDataDict = DatasetCatalog.get("datasetTrain")

for sample in random.sample(trainDataDict, 3):
    image = cv2.imread(sample["file_name"])
    visualizer = Visualizer(image[:, :, ::-1], metadata = trainMetadata, scale = 2)
    visualizedImage = visualizer.draw_dataset_dict(sample)
    cv2_imshow(visualizedImage.get_image()[:, :, ::-1])

### COCOTrainer zur Evaluierung

In [ ]:
class COCOTrainer(DefaultTrainer):

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):

      if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

      return COCOEvaluator(dataset_name, cfg, False, output_folder)

### Setup für Faster R-CNN Modell

In [ ]:
# Model Zoo Baselines for Object Detection
# https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("datasetTrain",)
cfg.DATASETS.TEST = ("datasetValidate",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001

cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4 # Anzahl der Klassen + 1 (3 Eier Klassen + 1)
cfg.MODEL.THINGCLASSES = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes

cfg.TEST.EVAL_PERIOD = 500

### Speichern der Modell-Konfiguration

In [ ]:
with open("model_cfg.pickle", 'wb') as f:
  pickle.dump(cfg, f, protocol = pickle.HIGHEST_PROTOCOL)

### Train the Model!

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)
trainer = COCOTrainer(cfg)
trainer.resume_or_load(resume = False)
trainer.train()

### Tensorboard für Ergebnisse

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

### Exportieren des Modells

In [ ]:
!zip -r /content/modelOutput.zip /content/coco_eval /content/output model_cfg.pickle
files.download("/content/modelOutput.zip")